In [35]:
import numpy as np
from scipy.optimize import linprog

# === Data ===
phones = ["A", "B", "C", "D"]
battery = np.array([19, 20, 10, 14])  # in hours
camera = np.array([500, 64, 32, 50])   # number of megapixels

# Normalize criteria (0-1 scale)
battery_norm = (battery - battery.min()) / (battery.max() - battery.min())
camera_norm = (camera - camera.min()) / (camera.max() - camera.min())

# Utilities matrix (each row = alternative, each col = criterion)
U_matrix = np.vstack([battery_norm, camera_norm]).T

# === Decision variables ===
# p = [p1, p2], weights on battery and camera
# Constraints: p1 + p2 = 1, p1, p2 >= 0

# Ranking constraints (B > A > D > C)
eps = 0.01
constraints = []
bounds_rhs = []

def add_constraint(better, worse):
    # U(better) - U(worse) >= eps
    row = better - worse
    constraints.append(-row)      # SciPy does A_ub * x <= b_ub
    bounds_rhs.append(-eps)

# Order: B > A > D > C
add_constraint(U_matrix[1], U_matrix[0])  # B > A
add_constraint(U_matrix[0], U_matrix[3])  # A > D
add_constraint(U_matrix[3], U_matrix[2])  # D > C

A_ub = np.array(constraints)
b_ub = np.array(bounds_rhs)

# Equality: p1 + p2 = 1
A_eq = np.array([[1, 1]])
b_eq = np.array([1])

# Objective: no real minimization needed (just feasibility)
# But linprog needs an objective, so minimize 0
c = np.array([0, 0])

res = linprog(c, A_ub=A_ub, b_ub=b_ub,
              A_eq=A_eq, b_eq=b_eq,
              bounds=[(0, None), (0, None)],
              method="highs")

print(res.message)
print()
print("Optimal weights:", res.x)
print("Battery weight:", res.x[0], "Camera weight:", res.x[1])

# Compute utilities for each phone
U_values = U_matrix @ res.x
for name, val in zip(phones, U_values):
    print(f"{name}: {val:.3f}")


Optimization terminated successfully. (HiGHS Status 7: Optimal)

Optimal weights: [1. 0.]
Battery weight: 1.0 Camera weight: 0.0
A: 0.900
B: 1.000
C: 0.000
D: 0.400
